In [18]:
import numpy as np
import pandas as pd
import boto3

file1=pd.read_csv('/Users/olgayangol/Desktop/git_proj/News_NLP_analysis/headlines_05092018.csv')
file2=pd.read_csv('/Users/olgayangol/Desktop/git_proj/News_NLP_analysis/headlines_05112018.csv')

In [13]:

Headlines = pd.concat([file1,file2],axis=0).reset_index()


In [14]:
Headlines.drop("index", axis = 1, inplace=True)

In [48]:
Headlines.head(10)
Headlines.shape

(235, 2)

In [16]:
Headlines['Headline'][0]

"Erdogan vows to quit when 'enough', sparks Twitter trend"

In [17]:
test=Headlines['Headline'][0]

In [20]:
nlp_engine=boto3.client(service_name="comprehend",region_name="us-east-1")

In [21]:
result=nlp_engine.detect_sentiment(Text=test,LanguageCode='en')

In [22]:
result

{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
   'content-length': '163',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Fri, 11 May 2018 18:37:07 GMT',
   'x-amzn-requestid': '4f16b8d2-554a-11e8-98e2-4d2e67fb8073'},
  'HTTPStatusCode': 200,
  'RequestId': '4f16b8d2-554a-11e8-98e2-4d2e67fb8073',
  'RetryAttempts': 0},
 'Sentiment': 'NEUTRAL',
 'SentimentScore': {'Mixed': 0.005120854824781418,
  'Negative': 0.10515657067298889,
  'Neutral': 0.8851268291473389,
  'Positive': 0.004595705308020115}}

In [25]:
print(result.keys())
print(result['Sentiment'])
sent_score_dict=result['SentimentScore']
print(sent_score_dict.keys())
print(sent_score_dict['Positive'])
print(sent_score_dict.items())

dict_keys(['Sentiment', 'SentimentScore', 'ResponseMetadata'])
NEUTRAL
dict_keys(['Positive', 'Negative', 'Neutral', 'Mixed'])
0.004595705308020115
dict_items([('Positive', 0.004595705308020115), ('Negative', 0.10515657067298889), ('Neutral', 0.8851268291473389), ('Mixed', 0.005120854824781418)])


In [26]:
key_phrase_res=nlp_engine.detect_key_phrases(Text=test,LanguageCode='en')

In [69]:
temp_result = key_phrase_res['KeyPhrases']
for item in temp_result:
    print(item['Text'])

Erdogan


In [33]:
nlp_input=list(Headlines['Headline'])

In [34]:
def nlp_sentiment(hl_list):
    sentiment_word=[]
    positive_score=[]
    neutral_score=[]
    negative_score=[]
    
    for hl in hl_list:
        result=nlp_engine.detect_sentiment(Text=hl,LanguageCode='en')
        sentiment_word.append(result['Sentiment']) #populate sentiment word list
        sent_score_dict=result['SentimentScore'] #create score dictionary
        positive_score.append(sent_score_dict['Positive']) #populate positive score list
        neutral_score.append(sent_score_dict['Neutral']) #populate negative score list
        negative_score.append(sent_score_dict['Negative']) #populate neutral score list
    return (sentiment_word,positive_score,neutral_score, negative_score)

In [35]:
sentiment_word,positive_score,neutral_score, negative_score=nlp_sentiment(nlp_input)

In [58]:
NLP_Headlines=Headlines

In [59]:
NLP_Headlines["Sentiment"]=sentiment_word
NLP_Headlines["Positive_score"]=positive_score
NLP_Headlines["Neutral_score"]=neutral_score
NLP_Headlines["Negative_score"]=negative_score


In [60]:
NLP_Headlines.head(10)

,Headline,Source,Sentiment,Positive_score,Neutral_score,Negative_score
0,"Erdogan vows to quit when 'enough', sparks Twi...",News24,NEUTRAL,0.004596,0.885127,0.105157
1,"Erdogan speech in Turkey backfires, leading to...",CBC.ca,NEUTRAL,0.015909,0.961792,0.019723
2,"I'll step aside when Turkey says 'enough', say...",euronews,NEUTRAL,0.072909,0.871536,0.044632
3,Has Erdogan accidentally gifted the Turkish op...,Kurdistan24,NEUTRAL,0.029603,0.923211,0.035578
4,Erdogan pledge to quit when 'enough' sparks Tw...,Gulf Times,NEUTRAL,0.009854,0.859170,0.122728
5,'That's enough' trends on Twitter after Turkey...,Daily Mail,NEUTRAL,0.008942,0.916661,0.068492
6,Turkish twitter hashtag 'Tamam' trending in op...,Middle East Eye,NEUTRAL,0.002121,0.995069,0.002612
7,"'Serbia, Turkey are building bridges of friend...",Brinkwire (press release),NEUTRAL,0.042800,0.942789,0.011974
8,Erdoğan's election manifesto,Daily Sabah,NEUTRAL,0.009923,0.942313,0.045632
9,How will the Turkish economy take shape in the...,Daily Sabah,NEUTRAL,0.003857,0.990248,0.005407


In [79]:
NLP_Headlines.to_csv('/Users/olgayangol/Desktop/git_proj/News_NLP_analysis/NLP_headlines.csv')

In [63]:
#!cat NLP_headlines.csv

In [74]:
def nlp_key_words(hl_list):
    key_words=[]
    
    for hl in hl_list:
        per_hl=[]
        result=nlp_engine.detect_key_phrases(Text=hl,LanguageCode='en')
        key_words_list=result['KeyPhrases']
        for key_word in key_words_list:
            per_hl.append(key_word['Text'])
        key_words.append(per_hl)
    
    return (key_words)



In [75]:
key_words=nlp_key_words(nlp_input)

In [76]:
len(key_words)

235

In [77]:
NLP_Headlines["Key_words"]=key_words

In [78]:
NLP_Headlines.head(10)

,Headline,Source,Sentiment,Positive_score,Neutral_score,Negative_score,Key_words
0,"Erdogan vows to quit when 'enough', sparks Twi...",News24,NEUTRAL,0.004596,0.885127,0.105157,[Erdogan]
1,"Erdogan speech in Turkey backfires, leading to...",CBC.ca,NEUTRAL,0.015909,0.961792,0.019723,"[Erdogan speech, Turkey, widespread]"
2,"I'll step aside when Turkey says 'enough', say...",euronews,NEUTRAL,0.072909,0.871536,0.044632,"[Turkey, Erdogan, Twitter]"
3,Has Erdogan accidentally gifted the Turkish op...,Kurdistan24,NEUTRAL,0.029603,0.923211,0.035578,"[Erdogan, the Turkish opposition, a catchy slo..."
4,Erdogan pledge to quit when 'enough' sparks Tw...,Gulf Times,NEUTRAL,0.009854,0.859170,0.122728,[Erdogan]
5,'That's enough' trends on Twitter after Turkey...,Daily Mail,NEUTRAL,0.008942,0.916661,0.068492,"[trends, Twitter, Turkey, President Erdogan, r..."
6,Turkish twitter hashtag 'Tamam' trending in op...,Middle East Eye,NEUTRAL,0.002121,0.995069,0.002612,"[Turkish twitter hashtag, 'Tamam, ' trending, ..."
7,"'Serbia, Turkey are building bridges of friend...",Brinkwire (press release),NEUTRAL,0.042800,0.942789,0.011974,"[Serbia, Turkey, bridges, friendship]"
8,Erdoğan's election manifesto,Daily Sabah,NEUTRAL,0.009923,0.942313,0.045632,[Erdoğan]
9,How will the Turkish economy take shape in the...,Daily Sabah,NEUTRAL,0.003857,0.990248,0.005407,"[the Turkish economy, shape, the post-election..."


In [ ]:
NLP_Headlines.to_csv('/Users/olgayangol/Desktop/git_proj/News_NLP_analysis/NLP_headlines.csv')